In [7]:
import glob
import os
import numpy as np
import pickle
import iris

In [15]:
def convert_dim2aux(cube, coord_name='time'):
    '''
    Converting an existing Dim coordinate to an Aux coordinate
    to assist with merging cubes with scalar Aux coordinates
    :param cube:
    :type cube:
    :param coord_name:
    :type coord_name:
    :return:
    :rtype:
    '''
    dim_coord = cube.coord(coord_name)
    aux_coord = iris.coords.AuxCoord(dim_coord.points, standard_name=dim_coord.standard_name,
                                     var_name=dim_coord.var_name, bounds=dim_coord.bounds,
                                     units=dim_coord.units)
    if cube.coords(coord_name):
        cube.remove_coord(coord_name)

    cube.add_aux_coord(aux_coord)
    return cube

In [123]:
pickle_file = '/scratch/hadpx/temp_for_moose_archive/merge.pkl'
infile = open(pickle_file,'rb')
precip_24hr_accum = pickle.load(infile)

In [130]:
data_array = np.ndarray(shape=(8, 36, 213, 178), dtype=float)
data_array.shape

(8, 36, 213, 178)

In [138]:
realiz_coord = iris.coords.DimCoord([int(mem) for mem in range(36)], standard_name='realization', var_name='realization')
#realiz_coord
forecast_period_coord = iris.coords.DimCoord([int(mem) for mem in range(0, 8*24,24)], standard_name='forecast_period', var_name='forecast_period')
forecast_period_coord


DimCoord(array([  0,  24,  48,  72,  96, 120, 144, 168]), standard_name='forecast_period', units=Unit('1'), var_name='forecast_period')

In [143]:
cube = iris.cube.Cube(data_array)
cube.add_dim_coord(forecast_period_coord, 0)
cube.add_dim_coord(realiz_coord, 1)
cube

Unknown (unknown),forecast_period,realization,--,--
Shape,8,36,213,178
Dimension coordinates,,,,
forecast_period,x,-,-,-
realization,-,x,-,-


In [98]:
dum = cube.coord('time').points[0]
if dum:
    print([dum-12, dum+12])

[455260.5, 455284.5]


In [144]:
precip_24hr_accum[1]

<iris 'Cube' of precipitation_amount / (kg m-2) (latitude: 213; longitude: 178)>

In [126]:
cubes = iris.cube.CubeList(precip_24hr_accum).merge()

DuplicateDataError: failed to merge into a single cube.
  Duplicate 'precipitation_amount' cube, with scalar coordinates realization=Cell(point=0, bound=None)

In [100]:
for cube in precip_24hr_accum:
    print(cube.coord('forecast_period'))

AuxCoord(array([0.]), bounds=array([[-21,   3]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([10.5]), bounds=array([[ 0, 24]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([22.5]), bounds=array([[24, 48]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([34.5]), bounds=array([[48, 72]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([46.5]), bounds=array([[72, 96]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([58.5]), bounds=array([[ 96, 120]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([70.5]), bounds=array([[120, 144]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([82.5]), bounds=array([[144, 168]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([0.]), bounds=array([[-21,   3]]), standard_name='forecast_period', units=Unit('hours'))
AuxCoord(array([10.5]), bounds=array([[ 0, 24]]), standard

In [12]:
if cubes.coord('forecast_period').bounds is not None:
    cubes.coord('forecast_period').bounds = None
    cubes.coord('forecast_period').guess_bounds()

ValueError: Cannot guess bounds for a coordinate of length 1.